In [ ]:
import os
import cv2
import pandas as pd
from shapely.geometry import Polygon, mapping
from osgeo import gdal
import rasterio
import geopandas as gpd
import numpy as np
import tqdm
import rasterio.mask
import torch


## Inicializar modelo entrenado deteccion de objetos basado en yolov7

In [ ]:
os.chdir("C:/Users/ASUS/Inteligencia_Artificial/yolov7")
from detect_Alberto_v2 import modelo
Modelo=modelo(weights="Modelos/best25.pt")

In [ ]:
# ##test
# dim=256
# testimage=r"train/images/zona_4_5_37.png"
# image=cv2.imread(testimage)
# image=cv2.resize(image,(256,256))
# verde=verificacion(image)
# vector2=[]
# vector2=Modelo.detect(opt_source=testimage,opt_conf_thres=0.1)
# angulo=0
# if (verde <70 and len(vector2)<20):
#     angulo = angulo+30
#     image_ro=image.copy()
#     M = cv2.getRotationMatrix2D((dim//2,dim//2), angulo, 1)
#     image_ro = cv2.warpAffine(image_ro, M, (dim,dim))
#     cv2.imwrite("test.png",image_ro)
#     vector1=Modelo.detect(opt_source="test.png",opt_conf_thres=0.1)
#     if len(vector2)<len(vector1):
#         vector2=vector1
#         image=image_ro
#     else:
#         vector2=Modelo.detect(opt_source=testimage,opt_conf_thres=0.1)
# vector2
# # imshow_detect(vector2xy(vector2,dim=256),image)

## Segmentar raster a partir de mascara de capa (normalizada)

In [ ]:
raster = r"C:\Users\ASUS\Desktop\Centro_Toluca\raster_cortado_centro.tif"
gdal_interpeter = gdal.Open(raster)
width = gdal_interpeter.RasterXSize
height = gdal_interpeter.RasterYSize
coordenadas_gdal = gdal_interpeter.GetGeoTransform()
minx = coordenadas_gdal[0]
miny = coordenadas_gdal[3] + width*coordenadas_gdal[4] + height*coordenadas_gdal[5] 
maxx = coordenadas_gdal[0] + width*coordenadas_gdal[1] + height*coordenadas_gdal[2]
maxy = coordenadas_gdal[3] 
minx,maxx,miny,maxy,"W",maxx-minx,"H",maxy-miny

In [ ]:
src_raster_path = raster
src=rasterio.open(src_raster_path)
H,W=src.shape

In [ ]:
dim=640
alto=int(np.floor(H/dim))
ancho=int(np.floor(W/dim))
nombres=[]
imshow=False
result=pd.DataFrame()
casas=[]
terreno=[]
for j in tqdm.tqdm(range(10)):#ancho
    for i in (range(1)):#alto
        # j=1
        label=raster.replace("\\","/").split("/")[-1][:-4]+"_"
        nameimg=label.lower()+str(i)+"_"+str(j)
        nombres.append(nameimg)
        cuadro=[]
        for k in range(2):
            for l in range(2):
                cuadro.append((minx+(maxx-minx)/ancho*(j+k),
                               maxy-(maxy-miny)/alto*(i+l),
                               0.0))
        cuadro=[cuadro[0],cuadro[1],cuadro[3],cuadro[2],cuadro[0]]
        shapes=[{"type":'Polygon','coordinates':[cuadro]}]
        vector=[]
        array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
        array=array.copy()
        four_images=[array[2],array[1],array[0]]
        stacked_images = np.stack(four_images, axis=-1)
        imagen_n=0
        imagen_n=stacked_images.copy()
        cv2.imwrite("cache.png",imagen_n)
        with torch.no_grad():
            vector=(Modelo.detect(opt_source="cache.png",opt_conf_thres=0.1))
     
        s=[]
        for v in vector:
            str_v=(str(v).replace("tensor(","").replace("[","").replace("]","").replace(".)","").replace(")","").replace("(","").replace("']","").replace("'","").strip().split(","))
            h,w=dim,dim
            x1 = int( float(str_v[1]) * w )
            y1 = int( float(str_v[2]) * h )
            xw = int( float(str_v[3]) * w /2)
            yw = int( float(str_v[4]) * h /2)
            start_point_im = ((x1 - xw), (y1 - yw))
            end_point_im   = ((x1 + xw), (y1 + yw))
            start_point_100 = ((x1 - xw)/w, (y1 - yw)/h)
            end_point_100   = ((x1 + xw)/w, (y1 + yw)/h)
            
            if str(str_v[0])=="0":
                tipo="casa"
            else:
                tipo="terreno"
            if int(xw)!=0 and int(yw)!=0  :
                s.append([tipo,start_point_im,end_point_im,start_point_100,end_point_100,nameimg])
        df_cache=pd.DataFrame(s,columns=["Tipo","start_point_im","end_point_im","start_point_100","end_point_100","imagen"])
        df_cache.drop_duplicates().reset_index(drop=True,inplace=True)
        proyecciones=shapes[0].get("coordinates")[0][:-1]
        min_y,min_x,z=np.min((proyecciones,proyecciones),axis=1)[0]
        max_y,max_x,z=np.max((proyecciones,proyecciones),axis=1)[0]
        min_y,min_x,max_y,max_x,proyecciones
        tipos=["casa","terreno"]
        for i in (range(len(df_cache))):
            coordenadas=[(df_cache["start_point_100"][i][1],df_cache["start_point_100"][i][0],df_cache["end_point_100"][i][1],df_cache["end_point_100"][i][0])]
            rectangulo=pd.DataFrame(coordenadas,columns=["x1","y1","x2","y2"])
            rectangulo["x1"]=max_x-rectangulo["x1"]*(max_x-min_x)
            rectangulo["y1"]=min_y+rectangulo["y1"]*(max_y-min_y)
            rectangulo["x2"]=max_x-rectangulo["x2"]*(max_x-min_x)
            rectangulo["y2"]=min_y+rectangulo["y2"]*(max_y-min_y)
            if df_cache["Tipo"][i]=="casa":
                casas.append(Polygon([(rectangulo["y1"][0],rectangulo["x1"][0]),(rectangulo["y2"][0],rectangulo["x1"][0]),(rectangulo["y2"][0],rectangulo["x2"][0]),(rectangulo["y1"][0],rectangulo["x2"][0])]))
            else:
                terreno.append(Polygon([(rectangulo["y1"][0],rectangulo["x1"][0]),(rectangulo["y2"][0],rectangulo["x1"][0]),(rectangulo["y2"][0],rectangulo["x2"][0]),(rectangulo["y1"][0],rectangulo["x2"][0])]))
        if imshow:
            for i in range(len(df_cache)):
                    if df_cache["Tipo"][i]=="casa":
                        cv2.rectangle(imagen_n,df_cache["start_point_im"][i],df_cache["end_point_im"][i],(0,255,0),2)
                    else:
                        cv2.rectangle(imagen_n,df_cache["start_point_im"][i],df_cache["end_point_im"][i],(0,0,255),2)
            cv2.imshow(nameimg,imagen_n)
            cv2.waitKey()
            cv2.destroyAllWindows()
        


In [ ]:
gdf_casas=gpd.GeoDataFrame(geometry=casas,crs=src.crs)
gdf_casas.set_crs=src.crs
gdf_casas.to_file(r"C:\Users\ASUS\Inteligencia_Artificial\yolov7/t_poligonos_casas_todo.shp")
gdf_casas["area"]=gdf_casas.area
gdf_casas=gdf_casas[gdf_casas["area"]<=gdf_casas["area"].mean()*2]
gdf_casas.to_file(r"C:\Users\ASUS\Inteligencia_Artificial\yolov7/t_poligonos_casas_1.shp")
gdf_terreno=gpd.GeoDataFrame(geometry=terreno,crs=src.crs)
gdf_terreno.set_crs=src.crs
gdf_terreno.to_file(r"C:\Users\ASUS\Inteligencia_Artificial\yolov7/t_poligonos_terreno_1.shp")


In [ ]:

33000*3/1360
alto,ancho,alto*ancho,(10*60+46)/60

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
gdf_casas=gpd.GeoDataFrame(geometry=casas,crs=src.crs)
gdf_casas.set_crs=src.crs
gdf_casas["area"]=gdf_casas.area
gdf_casas=gdf_casas[gdf_casas["area"]<=gdf_casas["area"].mean()*2]
gdf_casas
gdf_casas.to_file(r"C:\Users\ASUS\Inteligencia_Artificial\yolov7/nuevos_poligonos_casas.shp")

In [ ]:
ñ

In [ ]:
result.loc[0,"start_point_100"],result.loc[0,"end_point_100"]

In [ ]:
proyecciones=shapes[0].get("coordinates")[0][:-1]
np.min((proyecciones,proyecciones),axis=1)[0],np.max((proyecciones,proyecciones),axis=1)[0]

In [ ]:
proyecciones=shapes[0].get("coordinates")[0][:-1]
min_y,min_x,_=np.min((proyecciones,proyecciones),axis=1)[0]
max_y,max_x,_=np.max((proyecciones,proyecciones),axis=1)[0]
min_y,min_x,max_y,max_x,proyecciones

tipos=["casa","terreno"]
geometry=[]
for tipo in tipos:
    df_aux=df_cache[df_cache["Tipo"]==tipo].reset_index(drop=True)
    for i in tqdm.tqdm(range(len(df_aux))):
        coordenadas=[(df_aux["start_point_100"][i][1],df_aux["start_point_100"][i][0],df_aux["end_point_100"][i][1],df_aux["end_point_100"][i][0])]
        rectangulo=pd.DataFrame(coordenadas,columns=["x1","y1","x2","y2"])
        rectangulo["x1"]=max_x-rectangulo["x1"]*(max_x-min_x)
        rectangulo["y1"]=min_y+rectangulo["y1"]*(max_y-min_y)
        rectangulo["x2"]=max_x-rectangulo["x2"]*(max_x-min_x)
        rectangulo["y2"]=min_y+rectangulo["y2"]*(max_y-min_y)
        geometry.append(Polygon([(rectangulo["y1"][0],rectangulo["x1"][0]),(rectangulo["y2"][0],rectangulo["x1"][0]),(rectangulo["y2"][0],rectangulo["x2"][0]),(rectangulo["y1"][0],rectangulo["x2"][0])]))
# geometry
    gdf1=gpd.GeoDataFrame(geometry=geometry,crs=src.crs)
    gdf1.set_crs=src.crs
    gdf1.to_file(r"C:\Users\ASUS\Inteligencia_Artificial\yolov7/nuevos_poligonos_"+tipo+".shp")


In [ ]:
geometry[1]

In [ ]:
proyecciones=shapes[0].get("coordinates")[0][:-1]
miny,minx=proyecciones[0][:-1]
maxy,maxx=proyecciones[-1][:-1]

In [ ]:
vector=[]
array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
# array=np.moveaxis(out_image,0,-1)
four_images=[array[2],array[1],array[0]]
stacked_images = np.stack(four_images, axis=-1)
cv2.imwrite("cache.png",stacked_images)
vector=(Modelo.detect(opt_source="cache.png",opt_conf_thres=0.1))
vector


In [ ]:
s=[]
for v in vector:
    str_v=(str(v).replace("tensor(","").replace("[","").replace("]","").replace(".)","").replace(")","").replace("(","").replace("']","").replace("'","").strip().split(","))
    h,w=1024,1024
    x1 = int( float(str_v[1]) * w )
    y1 = int( float(str_v[2]) * h )
    xw = int( float(str_v[3]) * w /2)
    yw = int( float(str_v[4]) * h /2)
    start_point = ((x1 - xw), (y1 - yw) )
    end_point   = ((x1 + xw), (y1 + yw) )
    if str(str_v[0])=="0":
        tipo="casa"
    else:
        tipo="terreno"
    s.append([tipo,start_point,end_point])
resultado_nogeo=pd.DataFrame(s,columns=["Tipo","start_point","end_point"])
resultado_nogeo.drop_duplicates()

In [ ]:
cv2.imshow("s",stacked_images)
cv2.waitKey()

In [ ]:
for i in range(len(resultado_nogeo)):
    try:
        cv2.rectangle(stacked_images,resultado_nogeo["start_point"][i],resultado_nogeo["end_point"][i],(0,255,0),2)
    except:pass
cv2.imshow("s",stacked_images)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
resultado_nogeo["start_point"][i]